In [1]:
import pandas as pd
import numpy as np

from scipy.integrate import quad
from scipy.optimize import minimize


from multiprocessing import Pool
from functools import partial
import time
from datetime import datetime

from typing import Dict

import json
import logging
import sys

In [2]:
sample = json.load(open('data/snls_snia.json'))

df = pd.DataFrame(sample)

In [3]:
# Take Omega_k from args and calculate it after

def Dc(z, Omega_l, Omega_m, w):
    Omega_k = 1 - (Omega_l + Omega_m + 9.2e-5)
    def f(z, Omega_l, Omega_m, w):
        return (np.sqrt(Omega_l*(1 + z)**(-3 - 3*w) + Omega_m*(1 + z)**3 + Omega_k*(1 + z)**2 + 9.2e-5*(1 + z)**4))**(-1)
    return quad(f, 0, z, args = (Omega_l, Omega_m, w))[0]

def Dt(z, Omega_l, Omega_m, w):
    Omega_k = 1 - (Omega_l + Omega_m + 9.2e-5)
    if Omega_k > 0:
        return np.sin(np.sqrt(Omega_k)*Dc(z, Omega_l, Omega_m, w))/np.sqrt(Omega_k)
    elif Omega_k == 0:
        return Dc(z, Omega_l, Omega_m, w)
    elif Omega_k:
        return np.sinh(np.sqrt(abs(Omega_k))*Dc(z, Omega_l, Omega_m, w))/np.sqrt(abs(Omega_k))

def Dl(z, Omega_l, Omega_m, w):
    return (1 + z) * Dt(z, Omega_l, Omega_m, w)
               
def mu(z, Omega_l, Omega_m, w, Hubble):
    c = 3e5 #(km/s)
    return 5*np.log10(Dl(z, Omega_l, Omega_m, w)) + 25 + 5*np.log10(c/ Hubble)

def chi2_i(mu_o, sigma_o, z_obs, Omega_l, Omega_m, w, Hubble):
    return ((mu(z_obs, Omega_l, Omega_m, w, Hubble) - mu_o)**2)/sigma_o**2

def chi2(Omega_l, Omega_m, w, Hubble, df): 
    print(Omega_l, Omega_m, w, Hubble)
    values = []
    for idx in df.index:
        values.append(chi2_i(df.mu[idx], df.sigma[idx], df.z[idx], Omega_l, Omega_m, w, Hubble))
    return sum(values)

def ratio(Omega_l, Omega_m, w, Hubble, df, chi2null):
    return chi2(Omega_l, Omega_m, w, Hubble, df) - chi2null

def cross_product(*linspaces):
    # Create a list to hold the linspaces and their lengths
    linspaces_list = []
    lengths = []

    # Loop through each linspace and get its values and length
    for linspace in linspaces:
        linspaces_list.append(linspace)
        lengths.append(len(linspace))

    # Create a 2D array to hold the cross product
    cross_product_array = np.zeros((np.prod(lengths), len(linspaces)))

    # Loop through each value in the linspaces and add it to the cross product array
    for i in range(len(linspaces)):
        repeat = np.prod(lengths[i+1:])
        cross_product_array[:, i] = np.tile(np.repeat(linspaces[i], repeat), np.prod(lengths[:i]))

    return cross_product_array

In [4]:
def _zip_vars(fixed: dict, x) -> dict:
    full_parameters = ['Omega_l', 'Omega_m', 'w',  'Hubble']
    variable_parameters = []
    for param in full_parameters:
        if param not in list(fixed.keys()):
            variable_parameters.append(param)
            
    _vars = dict(zip(variable_parameters, x))
    return _vars

In [5]:
def h(x, fixed, df):
    """
    This is a wrapper for the chi2 function. 
    A workaround to the fact that scipy.optimize.minimize uses positional arguments
    
        Given a dict of fixed variables, this fuction enables minimize to pass an array of arguments
    """
    
     # Creates dictionary of variables to be optimized
    _vars = _zip_vars(fixed, x)

    # fixed['df'] = df
    g = partial(chi2, df = df, **fixed) # Creates partial function fixing variables set by `fixed`
        
    # Unzips `_vars` as named args to `g`
    return g(**_vars) 

In [13]:
config = json.load(open('config/likelihood_ratio_config.json'))

config['df'] = df

grid_size = config['grid_sizes'][0]

def _minimize(args):
    dict_kwargs = {
        'fun': h,
        'x0': args[2],
        'args': (args[0], args[1]),
        'method': 'Nelder-Mead',
        'jac': None,
        'hessp': None,
        'hess': None,
        'constraints': (),
        'tol': 1e-3,
        'callback': None,
        'options': None,
    }
    _min = minimize(**dict_kwargs)
    return _min.x

def generate_grid(config, grid_size):
    
    def generate_free_params(config, grid_size):
        _linspaces = []
        standard_x0 = {'Omega_l':0.74, 'Omega_m':0.26, 'w':-1,  'Hubble':70}
        for free in config['free']:
            _linspaces.append(np.linspace(config['grid'][free][0], config['grid'][free][1], grid_size))
        cross = cross_product(*_linspaces)
        for i in cross:
            fixed = {config['free'][0]: i[0], config['free'][1]: i[1]}
            yield (fixed, config['df'])
    
    # x0 = config['minimization_seed']
    
    # dict_kwargs = {'fun': h, 'x0': x0, 'method': 'Nelder-Mead', 'tol': 1e-6}
    
    
    # with Pool() as pool:
    #     results = pool.map(_minimize, generate_free_params(config, grid_size))
    #     pool.close()
    #     pool.join()
        
    # minimum = minimize(h, x0 = x0, args = (fixed, config['df']), method = 'Nelder-Mead', tol = 1e-6)
    # x0 = minimum.x

#     _vars = _zip_vars(fixed, minimum.x)

#     keys = ['Omega_l', 'Omega_m', 'w', 'Hubble']
#     values = [fixed.get(key,  _vars.get(key)) for key in keys]

    return None #tuple(values)

In [12]:
config['minimization_seed']

[0.74, 0.02, -1]

In [16]:
def generate_free_params(config):
    standard_x0 = {'Omega_l':0.74, 'Omega_m':0.26, 'w':-1,  'Hubble':70}
    _linspaces = []
    for free in config['free']:
        _linspaces.append(np.linspace(config['grid'][free][0], config['grid'][free][1], grid_size))
    cross = cross_product(*_linspaces)
    for i in cross:
        fixed = {config['free'][0]: i[0], config['free'][1]: i[1]}
        dict1 = {key: value for key, value in standard_x0.items() if key not in fixed}
        
        standard_x0_values = [value for value in dict1.values()]
        yield fixed, config['df'], standard_x0_values

In [19]:
for i in generate_free_params(config):
    print(i[2], (i[0], i[1]))

[0.74, -1] ({'Omega_m': 0.0, 'Hubble': 0.0},            mu    sigma        z
0    34.11747  0.19292  0.01543
1    34.08234  0.19033  0.01588
2    34.07026  0.19618  0.01615
3    34.40483  0.18699  0.01631
4    34.12864  0.18648  0.01645
..        ...      ...      ...
110  43.95315  0.29831  0.95000
111  43.62209  0.26844  0.96000
112  43.99927  0.31807  0.96100
113  43.94067  0.51326  0.98300
114  44.67224  0.54895  1.01000

[115 rows x 3 columns])
[0.74, -1] ({'Omega_m': 0.0, 'Hubble': 100.0},            mu    sigma        z
0    34.11747  0.19292  0.01543
1    34.08234  0.19033  0.01588
2    34.07026  0.19618  0.01615
3    34.40483  0.18699  0.01631
4    34.12864  0.18648  0.01645
..        ...      ...      ...
110  43.95315  0.29831  0.95000
111  43.62209  0.26844  0.96000
112  43.99927  0.31807  0.96100
113  43.94067  0.51326  0.98300
114  44.67224  0.54895  1.01000

[115 rows x 3 columns])
[0.74, -1] ({'Omega_m': 2.0, 'Hubble': 0.0},            mu    sigma        z
0    34.11747

In [9]:
# Optimizing every variable to find null hypothesis
# bounds and constrains
fixed = {}
# Omega_k = 1 - Omega_l - Omega_m - Omega_r
x0 = [0.76, 0.24, -1, 71] #['Omega_l', 'Omega_m', 'w',  'Hubble']
minimum = minimize(h, x0 = x0, args = (fixed, df), method = 'Nelder-Mead', tol = 1e-6, bounds = ((0,1), (0,1), (None, None), (0,None)), options = {'maxiter': 10000})

0.76 0.24 -1.0 71.0
0.798 0.24 -1.0 71.0
0.76 0.252 -1.0 71.0
0.76 0.24 -1.05 71.0
0.76 0.24 -1.0 74.55
0.7790000000000001 0.246 -1.025 67.45
0.7742500000000002 0.2445 -1.0187499999999998 69.225
0.7647499999999999 0.2415 -1.00625 72.775
0.7718750000000001 0.24375 -1.015625 70.1125
0.7849375000000001 0.247875 -0.9578125 70.55625
0.7787031250000002 0.24590625 -0.980859375 70.66718750000001
0.7372890624999999 0.250828125 -0.9982421875 70.38984375000001
0.7069335937499996 0.25624218750000005 -0.9973632812500002 70.08476562500002
0.7639335937500003 0.23824218749999998 -0.99736328125 70.08476562500002
0.7659003906250001 0.24936328124999996 -0.996044921875 69.62714843750001
0.7614750976562501 0.24234082031249998 -0.99901123046875 70.656787109375
0.7385832519531249 0.24167431640625003 -1.024261474609375 69.95476074218749
0.7185233154296873 0.23955834960937505 -1.045962524414063 69.59854736328123
0.7406776123046872 0.25105444335937505 -1.0212066650390625 70.47218017578123
0.7464916076660154 0.2

In [10]:
minimum.x

array([ 0.56166106,  0.        , -0.89896453, 70.15611471])

In [11]:
minimum

 final_simplex: (array([[ 0.56166106,  0.        , -0.89896453, 70.15611471],
       [ 0.56166122,  0.        , -0.89896499, 70.15611422],
       [ 0.561661  ,  0.        , -0.89896442, 70.15611499],
       [ 0.56166118,  0.        , -0.89896483, 70.15611441],
       [ 0.56166106,  0.        , -0.89896445, 70.15611556]]), array([110.94386128, 110.94386128, 110.94386128, 110.94386128,
       110.94386128]))
           fun: 110.94386128057178
       message: 'Optimization terminated successfully.'
          nfev: 360
           nit: 209
        status: 0
       success: True
             x: array([ 0.56166106,  0.        , -0.89896453, 70.15611471])